In [5]:
# Inputs for Factions Scraping
# Must be case specific and matches with factionDict below
factionTeam = "E4 Champions" # Make sure to check and see if the faction roster is set. Roster is a dict of "Team": [Roster]

# Change for each faction you import (in order of season, cycle)
# Note: depending on how each faction does, they might be in the same tier and as a result, you will need to manually go in and change it. :()
# If you are running a bout scrape, include all possible (1,1) entries like Gold and Diamond
cycleDictionary =  {"North America's First Cycle!": (0,1), "North America's Qualifiers": (0,2), "North America's Gold Cycle": (1,1),
                   "North America's Platinum Cycle": (1,2), "North America's Diamond Cycle":(1,3), "Promotions/Relegations":(1,1), "Global Melee - Platinum":(0,0), "Global Melee - Diamond":(0,0), "World Championship":(0,0), "North America's Emerald Cycle":(2,1)}


In [12]:
# Imports and URLs 
# DO NOT EDIT THIS PART UNLESS YOU ARE ADDING A TEAM INTO FACTION INFORMATION
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
import csv

# Faction Information. Roster is a dict of "Team": [Roster]. Case sensitive. Last updated: Emerald, 2022-05-23
factionRoster = {"Wing Attack": ['bhulbert', 'x3DxJMar159', 'Chellmic07', 'Motiks', 'xXRubixMasterXx', 'Rhyblet', 'WinstontheChamp', 'DarkerCatherine', 'pheon18', 'x3Dxconno7'], 
"TEAM OMEGALUL": ['FrostyACY','CustomApproach','Monstars50','SaadMunir','MagicMayson','EastsidePastor','MysticSparkle24','KimleyHorn','Jonathankelly','HououinK145'],
"Icy Wind": ['Wallower','MrKnollItAll','KevinSaludares','x3TheGOAT3x','jaysfan55','Dylan9497','DragOns1lk','FreakyBot9','Uberjudgement','ScrapinPete'],
"Evanstonks": ['LaprasLogan0', 'Warnerg', 'ProfSlade', 'Gundam95', 'leogeo0', 'basherbubbles', 'WindyQ','NUCULAAR','SageShadows','HollowPurple'], 
"Canadian Shieldon": ['Chumpoletta','Jamies550','JimmaBanks','obqlilpdo','Tangent444','FragginWagon','FreezingSunn','DubsOE','CoolTRNRBob', 'ThisistheSteve','Elleminator9000'],
"Chicago Stars": ['Balk88','Ajv28','SCHAPOCALYPSE','Abhinav7998','Burnobus','KeenBart','IFelici','howabunga','Zuoken', 'Uzoukwu'],
"Marillanders PvP": ['Swampyert','brnkbrinkbrnk','ShadowBrady','HotPoket777','Threeliters','SirJamz','EkkonomicaI','Huntress1079','Adakazam','LaprasLogan0'],
"Deoxys Alberta Forme": ['DaiLapCheurng','Sheldonk72','Garypuckinoak','Tribird3','friedriceballer','JBrandse','MisterBabushka','AShadyEquation','Callm3clutch','magmarman26'],             
"Stadium Elite": ['HouseStark93','Arrohh','JEngels21','Toshi9227','ValorAsh','Ilqm','ClashinCliff','IIIPranavIII','TINYBOY306','ShadowBrady'],
"Ann Arbor Arbok": ['MaiSakurajima2','ShinyDialga45','DijonTheDjinn','SolreLami196','2016tdouglas','ProfessorVitale','ShinyHunterBen','charydos','rott3np3anut','MasterRed'],
"Cool Cats": ['Twastell','CalebPeng','CochinoMum','PolymersUp','theChaoticAce','LindenRyu','Hsineerg','Auburnnnn','Ramberto777','dfhb1dwm'],
"DOJO Squad": ['Jjan11','Papacorn18','elodicolo','Poupsoup','th3six4ninja','Bucknaaasty','AlphaHaydonus','Kneelarmstrang','mattbeer87','TrentSzcz'],
"Team MichiGengar": ['AhmadZR', 'ashsaigh','TrentSzcz','CarissaV123','Red4Novak','TerdFergus0nn','Stodds92','Piscivore','HookedOnOnix19'], 
"Helmet Heroes": ['HoldinMcNuggets','0h0khaha','KakunaMattata42','AlphaFeeb','HangPJs','howardgarwong','recabecaaa','officialTT','Mhndztr'],
"AquaTail HungerForce": ['NNNNino468','CraigJames1','651Ryan','gastonagustin','OGPlayerOne','Leecifer','Bobonya5000','DD420x','YYdsPlusnull'],
"Madison Miltanks": ['NitroBuster','GottaKarchEmAll','Waterbug8','GBikekachu','SeawardObject9','WilliamPeter','GngrGirthQuake','HypnosProjectHQ','Logabibi'],
"SoCal Swablu": ['PsyRhino','Darthhferrett','R6RacingR6','DreFlames','MysticMamba24','19ZacAttack96','FrostFire14','CrystalBluPepsi','Queenofthecutie']}

PlatinumS1C2 = ["Team MichiGengar","Wing Attack", "TEAM OMEGALUL", "Icy Wind", "Helmet Heroes", "AquaTail HungerForce", "Madison Miltanks", "Evanstonks"]
DiamondS1C2 = ["Canadian Shieldon", "Chicago Stars", "Marillanders PvP", "Cool Cats", "Deoxys Alberta Forme", "DOJO Squad", "E4 Champions", "SoCal Swablu"]

PlatinumS1C3 = []
DiamondS1C3 = ["Wing Attack", "TEAM OMEGALUL", "Evanstonks", "Canadian Shieldon", "Chicago Stars", "Marillanders PvP", "Deoxys Alberta Forme", "E4 Champions"]

EmeraldS2C1 = ["Wing Attack", "TEAM OMEGALUL", "Icy Wind", "Evanstonks", "Canadian Shieldon", "Chicago Stars", "Deoxys Alberta Forme", "Stadium Elite", "Ann Arbor Arbok", "Cool Cats"]
# Function for scraping
def individualUserScrape(Username, queriedFaction):
    #Variables and Definitions
    URLBase = "https://sil.ph/"
    URL = URLBase + Username

    #Initializes web scrape
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, "html.parser")
    silphData = soup.find("div",id="networkAndAchievements")
    tournamentResults = silphData.find_all("div",class_="tournament")
    tournamentResultDataHolder = []
    for result in tournamentResults: 
        #Conditional formatting to check if factions bout and if so, sanitizes information for database entry
        boutInfo = result.find("h5", class_="tourneyName").text.strip()
        if not boutInfo.startswith('Bout'): continue

        #Check to see if given result is for the team listed (i.e. is this from a previous faction?)
        if not queriedFaction == result.find("a", class_="logo")["title"]: continue

        #Conditional formatting to change cupType into proper form, then into database entry
        cupType = result.find("h5", class_="cupType").text.strip().strip("★").strip()
        if "Great" in cupType: cupType = "Great"
        elif "Ultra" in cupType: cupType = "Ultra"
        elif "Master" in cupType: cupType = "Master"

        #This section needs to be updated to be more robust!!!    
        pattern = "Bout (.*): (.*)"
        parsedBoutInfo = re.findall(pattern,boutInfo)
        if parsedBoutInfo[0][1] == "North America's Emerald Cycle": 
            season = 2 
            cycle = 1
        else: continue 
        #season = cycleDictionary[parsedBoutInfo[0][1]][0]
        #cycle = cycleDictionary[parsedBoutInfo[0][1]][1]
        #Check to see if this particular bout was part of a promotion/relegation battle
        if parsedBoutInfo[0][1] == "Promotions/Relegations": boutNumber = 8 
        else: boutNumber = int(parsedBoutInfo[0][0])

        #Quick and dirty implementation of "Tier" designation -- should be combined with above at later date. 
        pattern = "Bout (.*): North America's (.*) Cycle"
        parsedBoutInfo = re.findall(pattern,boutInfo)
        if parsedBoutInfo == []: tier = ''
        else: tier = parsedBoutInfo[0][1]
        
        record = result.find(class_="win").find("h3", class_="value").text+'-'+result.find(class_="loss").find("h3", class_="value").text
        
        team = result.find_all(class_="pokemon")
        roster = []
        for mon in team:
            monName = mon["title"]
            #Conditional name edits to massage into proper form
            if monName == 'Armored Mewtwo': monName = 'Mewtwo-Armor'
            elif "Alolan" in monName: monName = monName.replace('Alolan ', '') + '-Alola'
            elif "Galarian" in monName: monName = monName.replace('Galarian ', '') + '-Galar'
            elif "Forme" in monName: 
                pattern = '\s\(*'
                result = re.split(pattern,monName)
                monName = result[0]+"-"+result[1]
            elif "Cloak" in monName: 
                pattern = '\s\(*'
                result = re.split(pattern,monName)
                monName = result[0]+"-"+result[1]
            elif "Size" in monName: 
                pattern = '\s\(*'
                result = re.split(pattern,monName)
                monName = result[0]+"-"+result[1]            
            elif "Castform" in monName:
                if "Snowy" in monName: monName = 'Castform-Snowy'
                elif "Rainy" in monName: monName = 'Castform-Rainy'
                elif "Sunny" in monName: monName = 'Castform-Sunny'
                elif "Normal" in monName: monName = 'Castform'
            if mon.find("img", class_="shadow"): monName = monName + '-S'
            roster.append(monName)
        tournamentResultDataHolder.append([tier, queriedFaction, Username, cupType, season, cycle, boutNumber, record] + roster)
    return tournamentResultDataHolder

In [ ]:
# Faction Member Scraping

In [13]:
# Run Factions Scraping for a given bout in a tier
# This will return erroneous results if the faction has been in the same tier -- blame Silph for not distinguishing the different cycles. 
boutData = []
exportedFileName = "Emerald Scrape S1C2B1"
desiredSeasons = range(2,2+1)# Starting season, ending season + 1, e.g. range(1,1+1) will get just season 1
desiredCycles = range(1,1+1) # Starting cycle, ending cycle +1
desiredBouts = range(1,1+1) # Starting bout, ending bout +1
desiredTier = EmeraldS2C1
for faction in desiredTier:
    for member in factionRoster[faction]:
        result = individualUserScrape(member,faction)
        for entry in result:
            if (entry[4] in desiredSeasons and entry[5] in desiredCycles and entry[6] in desiredBouts): boutData.append(entry)
with open(exportedFileName+".csv", 'w', newline='') as f:
    write = csv.writer(f)
    write.writerows([["Tier", "Team", "Player", "Format", "Season", "Cycle", "Bout", "Record", "Team #1", "Team #2", "Team #3", "Team #4", "Team #5", "Team #6"]])        
    write.writerows(boutData)

In [4]:
# Inputs and Code for Individual Scraping

result = individualUserScrape('IIIPranavIII', 'C-U Later')
with open('IIIPranavIII'+".csv", 'w', newline='') as f:
    write = csv.writer(f)
    write.writerows([["Tier", "Team", "Player", "Format", "Season", "Cycle", "Bout", "Team #1", "Team #2", "Team #3", "Team #4", "Team #5", "Team #6"]])        
    write.writerows(result)